# Type Hints

Python Version: 3.5 but with many improvements added in the following releases.

[Official Docs](https://docs.python.org/3/library/typing.html)

[Mypy cheatsheet](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html)

Python supports type hints which say what type of value a variable or function parameter is. They are not enforced by the Python interpreter but can be used by static type checkers like Mypy or Pylance to catch errors.

## Why use type hints?

Type hints make catching errors easier and code more readable. For instance, a function signature like `def convert(date)` can be confusing on its own; is date a string, a datetime object, or something else? Type hints concisely show you how a program expects to be used.

I think they're at their best catching unexpected return values from functions. For instance, you might have a hole in your logic that results in a function returning `None` (the value returned when you do not explicitly `return` anything) and then later consume that return value but not account for it being `None`. Another way they're immensely helpful is that type checkers _known the types and return values of everything in the standard library_ and most things in imported modules. They check that your understanding of others' code is correct.

When I started using type checking, one of the most common warnings I saw on our Django website was a lot of code which looks like `db.objects().first()` where I assumed I had an object but in fact this method can return `None` if no objects have been created in the database yet. Type hints know the expect return values of these `db` methods and catch such errors.

Another super common error in my code: the iterator returend by pymarc's `MARCReader` does not always return a `Record` object, sometimes it returns `None`:

In [ ]:
from pymarc import MARCReader

with open('example.mrc', 'rb') as fh:
    reader = MARCReader(fh)
    for record in reader:
        # record could be None so we need this if condition
        if record:
            dosomething(record)



## But how do they work?

Type hints for variables and parameters consistent of a colon followed by the type. Type hints for functions' return values are indicated by an arrow `->`. Types are the same values you see when you call `type()` on a value, such as `int`, `str`, `list`, `dict`, etc.

In [1]:
from random import randint

# basic type hints
name: str = "Eric"
age: int = 87

# python "knows" randint returns an int so this works
def random_age() -> int:
    return randint(0, 100)

age = random_age()

def greeting(fname: str) -> str:
    return f"Hello, {fname}"

print(greeting(age)) # warning! age is an int, not a str

Hello, 23


The above is an excellent example because the `f` string returned by `greeting` actually doesn't care that it has an `int` inside it, it coerces the `int` to a string. So there's no error thrown, but our function wasn't used in the intended way! The type hints are catching something that is technically valid code, which does not raise an exception, but is not what we intended.

When you annotate something as a list, you can also specify what types of values are in the list. For instance, `List[int]` is a list of integers.

When declaring type hints for dicts, you declare the type of the keys and the values. For instance, `Dict[str, int]` is a dictionary where the keys are strings and the values are integers.



In [2]:
english_numbers: dict[int, str] = {
    1: "one",
    2: "two",
    3: "three",
}
number_words: list[str] = list(english_numbers.values())

Why if my type isn't one of the core ones included in the global namespace? You can import classes from modules to use in your type hints. There is also a `typing` module with many important utilities.

In [4]:
from os import stat, stat_result # stat_result is a specific object structure
from pathlib import Path # we are only using Path in our type hint

def stat_doc(doc: Path) -> stat_result:
    # The "/" operator wouldn't work without a Path argument, can't use with 2 strings
    doc = "/Users/ephetteplace/Documents" / doc
    return stat(doc)

stat_doc(Path("DataCite-MetadataKernel_v4.4.pdf"))

os.stat_result(st_mode=33204, st_ino=2413209, st_dev=16777234, st_nlink=1, st_uid=502, st_gid=20, st_size=972986, st_atime=1710467870, st_mtime=1685738676, st_ctime=1701375669)

## Your Own Classes

Have a custom structure which your code uses over and over? Type hints give you a concrete reason to define a class for it. There was a discussion a while back in the python4lib Slack channel about when to start using object-oriented programming and folks felt more comfortable writing mostly functions. That makes perfect sense, but type hints can provide an incentive to create a little more structure and abstract commons patterns from your data.

In [ ]:
from datetime import date

class Student():
    def __init__(self, name: str, grad_date: str|None) -> None:
        self.name: str = name
        self.graduation_date: date | None = date.fromisoformat(grad_date) if grad_date else None

    @property
    def is_alumni(self) -> bool:
        # can't do this grad date math if it's a string
        return self.graduation_date is not None and self.graduation_date < date.today()

person = {
    "name": "Eric",
    "graduation_date": "2022-05-15"
}

# this raises an error because student is a dict, not a Student object with the
# is_alumni property calculated from their graduation date
if person.is_alumni:
    print(f"ask {person.name} for a donation")

person = Student(**person)

if person.is_alumni:
    print(f"ask {person.name} for a donation") # this works!

To do:
- Any and working around errors (e.g. from external libraries you can't control)
- Optional
- Union types
- Literal